<a href="https://colab.research.google.com/github/vervainalthor/Data-analytics-for-medium/blob/main/Who_is_giving_me_less_money.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LIFE TIME VALUE PREDICTION Prediction With RFM

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from datetime import datetime
from scipy.stats import skew  # for some statistics why?
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor # these are high powered regression models i know nothing about
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [2]:
dfacct = pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/account.csv', sep=";", header=0)

dfcard = pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/card.csv', sep=";", header=0)
dfclient = pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/client.csv', sep=";", header=0)

dfdistrict = pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/district.csv', sep=";", header=0)

dfdisp = pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/disp.csv', sep=";", header=0)

dftrans=pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/trans.csv', sep=";",
                    header=0)
dfloan=pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/loan.csv', sep=";",
                    header=0)
dforder=pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/order.csv', sep=";",
                    header=0)

<ipython-input-2-29ad8d1298f2>:10: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dftrans=pd.read_csv('/content/drive/MyDrive/Peronal Customer Segementation and churn/lpetrocelli-czech-financial-dataset-real-anonymized-transactions/trans.csv', sep=";",


#Renaming columns and replacing Czech categories with an english equivalent

## Account table

In [3]:
dfacct['date']= pd.to_datetime(dfacct['date'], format='%y%m%d')

In [4]:
dfacct.loc[dfacct['frequency'].isin(['POPLATEK MESICNE']), 'frequency'] = 'monthly issuance'

In [5]:
dfacct.loc[dfacct['frequency'].isin(['POPLATEK TYDNE']), 'frequency'] = 'weekly issuance'

In [6]:
dfacct.loc[dfacct['frequency'].isin(['POPLATEK PO OBRATU']), 'frequency'] = 'post tranissuance'

In [7]:
dfacct[dfacct['frequency'].str.contains('post')]

,account_id,district_id,frequency,date
6,1539,1,post tranissuance,1993-01-03
14,485,6,post tranissuance,1993-01-04
130,1965,57,post tranissuance,1993-02-08
148,539,13,post tranissuance,1993-02-16
175,1050,57,post tranissuance,1993-02-24
...,...,...,...,...
4269,3103,1,post tranissuance,1997-09-21
4397,2211,61,post tranissuance,1997-11-10
4409,3156,1,post tranissuance,1997-11-16
4425,1523,52,post tranissuance,1997-11-24


In [8]:
dfacct.head()

,account_id,district_id,frequency,date
0,576,55,monthly issuance,1993-01-01
1,3818,74,monthly issuance,1993-01-01
2,704,55,monthly issuance,1993-01-01
3,2378,16,monthly issuance,1993-01-01
4,2632,24,monthly issuance,1993-01-02


In [9]:
dfdisp= dfdisp[dfdisp["type"].str.contains("OWNER")]

In [10]:
dfdisp.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
3,4,4,3,OWNER
5,6,6,4,OWNER
6,7,7,5,OWNER


##Card ownership

In [11]:
dfcard['issued']= pd.to_datetime(dfcard['issued'], format='%y%m%d %H:%M:%S') 

In [12]:
# Extract only the date component
dfcard['issued'] = dfcard['issued'].dt.date

In [13]:
dfcard.head()

,card_id,disp_id,type,issued
0,1005,9285,classic,1993-11-07
1,104,588,classic,1994-01-19
2,747,4915,classic,1994-02-05
3,70,439,classic,1994-02-08
4,577,3687,classic,1994-02-15


## Demograhic key details

In [14]:
dfdistrict = dfdistrict.rename(columns={'A1': 'district_code', 'A2': 'district_name', 'A3': 'region',
                                        'A4': 'inhabitant_count', 'A8': 'inhabitants<10kcount', 'A12': 'unemployment rate_95',
                                        'A5': 'inhabitants<499count', 'A9.': 'citiescount', 'A13': 'unemployment rate_96',
                                        'A6': 'inhabitants>5h<2kcount', 'A10': 'urban_inhabitantratio', 'A14': 'entrepreneurs_per_1000','A15': 'committedcrimes_95',
                                        'A7': 'inhabitants<2k<10kcount', 'A11': 'average_salary', 'A16': 'committedcrimes_95'})


# CREATING THE RFM TABLE WITH TRANSACTION TABLE

* renaming content

In [15]:
dftrans['date']= pd.to_datetime(dftrans['date'], format='%y%m%d') 

In [16]:
dftrans.loc[dftrans['type'].isin(['PRIJEM']), 'type'] = 'credit'

In [17]:
dftrans.loc[dftrans['type'].isin(['VYDAJ']), 'type'] = 'debit'

In [18]:
dftrans['operation'] = dftrans['operation'].replace({'VYBER KARTOU': 'creditcardwitdrawal','VKLAD':'credit_in_cash', "PREVOD Z UCTU":'anotherbankcredit'
,"VYBER":'cashdebit',"PREVOD NA UCET":'tranferdebit'})

In [19]:
dftrans['operation'].unique()

array(['credit_in_cash', 'anotherbankcredit', 'cashdebit', nan,
       'tranferdebit', 'creditcardwitdrawal'], dtype=object)

In [20]:
dftrans.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,1993-01-01,credit,credit_in_cash,700.0,700.0,NaN,NaN,NaN
1,171812,576,1993-01-01,credit,credit_in_cash,900.0,900.0,NaN,NaN,NaN
2,207264,704,1993-01-01,credit,credit_in_cash,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,1993-01-01,credit,credit_in_cash,600.0,600.0,NaN,NaN,NaN
4,579373,1972,1993-01-02,credit,credit_in_cash,400.0,400.0,NaN,NaN,NaN


#      RFM ANALYSIS DATA GATHERING
* RFM for traditional segmentation 
* RFM for the purpose of Customer lifetime valueprediction 
* RFM for customer lifetime value Predictions

In [21]:
dftrans['date'].max()

Timestamp('1998-12-31 00:00:00')

In [22]:
dftrans['date'].min()

Timestamp('1993-01-01 00:00:00')

# M-CUSTOMER VALUE FOR EVERY TWO YEARS


In [23]:
df93to94M= dftrans[ dftrans['date'] < pd.Timestamp('1995-01-01')] #train
df95to96M= dftrans[( dftrans['date'] >= pd.Timestamp('1995-01-01')) & ( dftrans['date'] < pd.Timestamp('1997-01-01'))] #test
df97to98M= dftrans[( dftrans['date'] >= pd.Timestamp('1997-01-01')) & ( dftrans['date'] < pd.Timestamp('1999-01-01'))] #holdout

In [24]:
df93to94M.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,1993-01-01,credit,credit_in_cash,700.0,700.0,NaN,NaN,NaN
1,171812,576,1993-01-01,credit,credit_in_cash,900.0,900.0,NaN,NaN,NaN
2,207264,704,1993-01-01,credit,credit_in_cash,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,1993-01-01,credit,credit_in_cash,600.0,600.0,NaN,NaN,NaN
4,579373,1972,1993-01-02,credit,credit_in_cash,400.0,400.0,NaN,NaN,NaN


In [25]:
df93to94M = df93to94M.groupby('account_id').sum()
df93to94M.drop(['trans_id', 'balance','account'], axis=1, inplace=True)
df93to94M.reset_index(inplace=True)
df93to94M.head()


<ipython-input-25-1f02cd138231>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df93to94M = df93to94M.groupby('account_id').sum()


,account_id,amount
0,2,953763.7
1,6,20907.0
2,9,182923.8
3,15,545827.0
4,18,1238112.0


In [26]:
df95to96M = df95to96M.groupby('account_id'	).sum()
df95to96M .drop(['trans_id', 'balance','account'], axis=1, inplace=True)
df95to96M.reset_index(inplace=True)
df95to96M.head()


<ipython-input-26-45e82376f14e>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df95to96M = df95to96M.groupby('account_id'	).sum()


,account_id,amount
0,1,186600.3
1,2,1087332.5
2,4,88898.1
3,6,313157.8
4,7,41505.4


In [27]:
df97to98M = df97to98M.groupby('account_id'	).sum()
df97to98M.drop(['trans_id', 'balance','account'], axis=1, inplace=True)
df97to98M.reset_index(inplace=True)
df97to98M.head()


<ipython-input-27-9930bcd06f94>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df97to98M = df97to98M.groupby('account_id'	).sum()


,account_id,amount
0,1,188574.2
1,2,1110383.1
2,3,295021.8
3,4,262073.3
4,5,166881.4


#left join all three groups and replace nans with 0 to indicate churn

In [28]:
df_value = pd.merge(df93to94M, df95to96M, on='account_id', how='left')
df_value  = pd.merge(df_value,df97to98M, on='account_id', how='left')


In [ ]:
df_value.isnull().sum()
df_value.fillna(0)


In [34]:
df_value.rename(columns={'amount_x': 'Value2', 'amount_y': 'Value1', 'amount': 'currentvalue'}, inplace=True)
df_value['overallchange'] =df_value['currentvalue'] / df_value['Value2']
df_value['yoychng1'] =df_value['Value1'] / df_value['Value2']
df_value['yoychng'] =df_value['currentvalue'] / df_value['Value1']

In [35]:
df_value.head()

,account_id,Value2,Value1,currentvalue,overallchange,yoychng1,yoychng
0,2,953763.7,1087332.5,1110383.1,1.164212,1.140044,1.021199
1,6,20907.0,313157.8,313502.3,14.995088,14.978610,1.001100
2,9,182923.8,232996.3,280514.0,1.533502,1.273734,1.203942
3,15,545827.0,903959.3,879786.6,1.611841,1.656128,0.973259
4,18,1238112.0,675867.3,1814719.0,1.465715,0.545885,2.685023


In [36]:
df_value.to_csv('change.csv', index=False, header=True)